# Variant Filtering

In the next series of commands we will learn how to extract information from VCFs and how to filter the raw calls. We will use the bcftools commands again. Most of the commands accept the `-i, --include` and `-e, --exclude` options [https://samtools.github.io/bcftools/bcftools.html#expressions](https://samtools.github.io/bcftools/bcftools.html#expressions) which will be useful when filtering using fixed thresholds. We will estimate the quality of the callset by calculating the ratio of transitions and transversions [https://en.wikipedia.org/wiki/Transversion](https://en.wikipedia.org/wiki/Transversion).

When drafting commands, it is best to build them gradually. This prevents errors and allows you to verify that they work as expected. Let's start with printing a simple list of positions from the VCF using the bcftools query command [https://samtools.github.io/bcftools/bcftools.html#query](https://samtools.github.io/bcftools/bcftools.html#query) and pipe through the head command to limit the printed output to the first few lines:

In [ ]:
bcftools query --format 'POS=%POS\n' out.vcf | head

As you can see, the command expanded the formatting expression `POS=%POS\n` in the following way: for each VCF record the string POS= was copied verbatim, the string `%POS` was replaced by the VCF coordinate stored in the POS column, and then the newline character `\n` ended each line. (Without the newline character, positions from the entire VCF would be printed on a single line.)

Now add the reference and the alternate allele to the output. They are stored in the REF and ALT column in the VCF, and let's separate them by a comma:

In [ ]:
bcftools query -f'%POS %REF,%ALT\n' out.vcf | head

In the next step add the quality (%QUAL), genotype (%GT) and sequencing depth (%AD) to the output. Note that FORMAT tags must be enclosed within square brackets [...] to iterate over all samples in the VCF. (Check the Extracting per-sample tags section in the manual https://samtools.github.io/bcftools/howtos/query.html for a more detailed explanation why the square brackets are needed.)

In [ ]:
bcftools query -f'%POS %QUAL [%GT %AD] %REF %ALT\n' out.vcf | head

Now we are able to quickly extract important information from the VCFs. Now let's filter rows with QUAL smaller than 30 by adding the filtering expression --exclude 'QUAL<30' or --include 'QUAL>=30' like this:

In [ ]:
bcftools query -f'%POS %QUAL [%GT %AD] %REF %ALT\n' -i'QUAL>=30' out.vcf | head

Now compare the result with the output from the previous command, were the low-quality lines removed? 

In the next step limit the output to SNPs and ignore indels by adding the type="snp" condition to the filtering expression. Because both conditions must be valid at the same time, we request the AND logic using the && operator:

In [ ]:
bcftools query -f'%POS %QUAL [%GT %AD] %REF %ALT\n' -i'QUAL>=30 && type="snp"' out.vcf | head   

## Exercises

**Q1:** Can you print SNPs with QUAL bigger than 30 and require at least 25 alternate reads in the AD tag? 

Remember, the first value of the AD tag is the number of reference reads, the second is the number of alternate reads, therefore you will need to query the second value of the AD tag. The first value can be queried as AD[0] and the second as AD[1] (the allele indexes are zero-based).  In case of FORMAT fields, also the queried sample must be selected as  AD[sample:subfield] . Therefore add to the expression the condition AD[0:1] >= 25 to select the first (and in our case the only one) sample or  AD[*:1] >= 25 to select any sample for which the condition is valid.

Now we can filter our callset. In order to evaluate the quality, we will use bcftools stats to calculate the ratio of transitions vs transversions. We start by checking what is the ts/tv of the raw unfiltered callset. The `stats` command produces a text output, we extract the field of interest as follows:

In [ ]:
bcftools stats out.vcf | less
bcftools stats out.vcf | grep TSTV
bcftools stats out.vcf | grep TSTV | cut -f5

**Q2:** Calculate ts/tv of the set filtered as above by adding -i 'QUAL>=30 && AD[*:1]>=25' to the bcftools stats  command. (Here the asterisk followed by a colon tells the program to apply the filtering to all samples. At least one sample must pass in order for a site to pass.) After applying the filter, you should observe an increased ts/tv value.


**Q3:** Can you do the reverse and find out the ts/tv of the removed sites? Use the `-e` option instead of `-i`. The ts/tv of the removed low-quality sites should be lower.


**Q4:** The test data come from an inbred homozygous mouse, therefore any heterozygous genotypes are most likely mapping and alignment artefacts. Can you find out what is the ts/tv of the heterozyous SNPs? Do you expect higher or lower ts/tv? Use the filtering expression `-i 'GT="het"'` to select sites with heterozygous genotypes.

Another useful command is `bcftools filter`  which allows to "soft filter" the VCF: instead of removing sites, it can annotate the FILTER column to indicate sites which fail. Apply the above filters ('QUAL>=30 && AD[*:1]>=25') to produce a final callset, adding also the `--SnpGap` and the `--IndelGap` option to filter variants in close proximity to indels:

In [ ]:
bcftools filter -s LowQual -i'QUAL>=30 && AD[*:1]>=25' -g8 -G10 out.vcf -o out.flt.vcf

## Variant normalization

The same indel variant can be represented in different ways. For example, consider the following 2bp deletion. Although the resulting sequence does not change, the deletion can be placed at two different positions within the short repeat:

           12345
           TTCTC
    POS=1  T--TC
    POS=3  TTC--

In order to be able to compare indels between two datasets, we left-align such variants.

**Q5:** Use the bcftools norm command to normalize the filtered callset. Note that you will need to provide the `--fasta-ref` option. Check in the output how many indels were realigned.

Now continue to the next section of the tutorial: [Multi-sample variant calling](multi-sample-calling.ipynb)